In [1]:
# Program Purpose: Find Correlation between target column and all other columns. Automates statistical 
# tests of correlation as well as allows any analyst to find association between all types of variables 
# without extensive statistical experience.

# First, the program takes an input of a excel file or a csv file and
# then have the user input a target column. This target will then be compared to all of the 
# other columns through the use of statistical tests. These tests will then determine correlation
# between the target and the columns. The program classifies each of these correlations as
# either large, medium, small, or none. The program outputs a list of all of these correlations
# based on the category that they were assigned. 

import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt


# CSV file, path to user input that will be used in correlation analysis
df = pd.read_csv('C:/Users/Ted/Documents/Data/apple_quality.csv')                          
#df = pd.read_excel('C:/Users/Ted/Documents/Data/FinancialSample.xlsx')
target = 'Quality'        

# Checking to ensure that the target that has been entered is a valid column name
if target not in df.columns:
    raise ValueError(f"The target column '{target}' does not exist in the dataframe. Please enter a valid column name.")

# -------------------------------------------------------------------------------------
# Settings
    
bins = int(np.sqrt(len(df[target])))         # Default: int(np.sqrt(len(df[target])))
max_unique_values_threshold = 50             # Default: 50. Only for categorical data. This is the number of categories that an object is limited to having, chi2 gets messed up when there are too many
showCrosstab = False                         # Chi2 
showBinnedCrosstab = False                   # Binned Chi2, Depending on the dataset, these crosstabs can be very large and hard to understand but can still glean information from it if interested
showGroupMeans = False                       # ANOVA    

# -------------------------------------------------------------------------------------

# Here the settings section ends, and the program begins 

# Categories for each degree of correlation, each variable will be put into a bucket
correlation_categories = {
    "large": [],
    "medium": [],
    "small": [],
    "none": [],
    "unable": []
}


# for every column that is not the target column, check the dtype of that 
# column and the target column and see if they are both object
for column in df.columns:
    if 'Unnamed' not in column and target != column:
        
        # this is used to determine if the object (categorical) target or column have too many unique values
        # if this is the case, the tests will be unaccurate and should not be used, therefore they are put in the unable column
        if (df[target].dtype == 'object' and df[target].nunique() > max_unique_values_threshold):
            print(f"Target '{target}' has {df[column].nunique()} unique values, which is more than the threshold of {max_unique_values_threshold}.")
            print("Attempts at correlation would yield unusable results")
            print()
            #correlation_categories["unable"].append(column)
            break 
        elif (df[column].dtype == 'object' and df[column].nunique() > max_unique_values_threshold):
            #print(f"Column '{column}' has {df[column].nunique()} unique values, which is more than the threshold.")
            correlation_categories["unable"].append(column)
            continue
            
        
        # object vs object: chi2 and Cramer's V (bias corrected)
        # this test compares observed and expected frequencies of the crosstab
        if df[target].dtype == 'object' and df[column].dtype == 'object': 
            
            ct = pd.crosstab(df[target], df[column])
                       
            chi2, p, dof, expected = stats.chi2_contingency(ct)
            critical_value = stats.chi2.ppf(1 - 0.05, dof)
            
            r, k = ct.shape                                      # Number of rows and columns
            n = np.sum(ct.values) 
            k_corrected_term = ((k - 1)**2) / (n - 1)            # Bias correction terms
            r_corrected_term = ((r - 1)**2) / (n - 1)
            k_corrected = k - k_corrected_term                   # Corrected number of rows and columns for bias correction
            r_corrected = r - r_corrected_term
                    
            if p < 0.05:
                if chi2 > critical_value:
                    
                    print('Test: Chi-Square (Χ²) and Cramers V')
                    
                    print(f'Target: {target} ({df[target].dtype})')
                    print(f'Column: {column} ({df[column].dtype})')
                    print()
                    
                    cramers_v = np.sqrt((chi2 / n) / min(k_corrected - 1, r_corrected - 1))
    
                    if cramers_v > 0.5:
                        doc = 'Large'
                        correlation_categories["large"].append(column)
                    elif cramers_v > 0.3:
                        doc = 'Medium'
                        correlation_categories["medium"].append(column)
                    elif cramers_v > 0.1:
                        doc = 'Small'
                        correlation_categories["small"].append(column)
                    else:
                        doc = 'None'
                        correlation_categories["none"].append(column)

                    print(f'chi2: {chi2:.3f}')
                    print(f'cv: {critical_value:.3f}')          # this is the value that chi squared must be below or above to determine if it is high or low
                    print(f'p: {p:.3f}')
                    print(f'Cramers V: {cramers_v:.3f}')        # note that Cramers V works but is not suited towards 2x2 contingency tables
                    
                    print()    
                    print('Correlation:', doc)
                    
                    if showCrosstab == True:
                        df_temp = df[[target, column]].copy()
                        
                        if (df_temp[column].nunique() < 10):
         
                            # Rename columns for easier readability in crosstab output
                            df_temp.rename(columns={target: 'Target', column: 'Column'}, inplace=True)

                            # Now use df_temp for crosstab
                            ct = pd.crosstab(df_temp['Target'], df_temp['Column'])

                            print()
                            print('Crosstab:')
                            print()
                            print(ct)
                            print()
                        else:
                            print()
                            print('Too many groups, choosing to not display crosstab')
                            
                    
                    #print('p value is below the significance threshold of 0.05')
                    #print('chi2 value is greater than critical value')
                    #print('the difference between observed and expected frequencies is too large to be attributed to chance')

                    print()
                    print('---------------------------------------------------------')
                    print()
                    
                else: 
                    correlation_categories["none"].append(column)
                    #print('No Correlation')
                    #print()
                    #print('p value is above the significance threshold of 0.05')
                    #print('chi2 value is less than critical value')
                    #print('the difference between observed and expected frequencies is too small and could be attributed to chance')
                    
            else:
                correlation_categories["none"].append(column)
                #print('No Correlation')
                #print()
                #print('p value is above the significance threshold of 0.05')
       

    

        # numerical vs numerical: pearsons or spearmans
        # this test that measures the strength and direction of association between two variables
        elif (df[target].dtype == 'float64' or df[target].dtype == 'int64') and (df[column].dtype == 'float64' or df[column].dtype == 'int64'):
            
            
            x, p = stats.spearmanr(df[target], df[column])       # can use either pearsonr or spearmanr depending on how the data is distributed             
            
            #print(df[target])
            #print(df[column])
            #print(target, column, x, p)
            
            if p < 0.05:
                
                print('Test: Spearmans Correlation Coefficient')
                
                if x >= 0.5:
                    doc = 'Large Positive'
                    correlation_categories["large"].append(column)
                elif x >= 0.3:
                    doc = 'Medium Positive'
                    correlation_categories["medium"].append(column)
                elif x >= 0.1:
                    doc = 'Small Positive'
                    correlation_categories["small"].append(column)
                elif x < 0.1 and x > -0.1:
                    doc = 'None'
                    correlation_categories["none"].append(column)
                elif x <= -0.1 and x > -0.3:
                    doc = 'Small Negative'
                    correlation_categories["small"].append(column)
                elif x <= -0.3 and x > -0.5:
                    doc = 'Medium Negative'
                    correlation_categories["medium"].append(column)
                elif x <= -0.5 and x >= -1:
                    doc = 'Large Negative'
                    correlation_categories["large"].append(column)

                print(f'Target: {target} ({df[target].dtype})')
                print(f'Column: {column} ({df[column].dtype})')
                print()

                print(f'x: {x:.3f}')
                print(f'p: {p:.3f}')

                print()
                
                print('Correlation:', doc)
                #print('p value is below the significance threshold of 0.05')    
                
                print()
                print('---------------------------------------------------------')
                print()
                
            else:
                correlation_categories["none"].append(column)
                #print('No Correlation')
                #print('Reason: p value is above the significance threshold of 0.05')
              

            
            
        # object vs. numerical: ANOVA
        # independent variables: columns. dependent variable: target
        # this test uses expected means of each group, and checks to see if there is a 
        # significant difference between these group means
        # There can be no technical 'correlation' between numerical vs categorical, only the result is something very similar 
        elif ((df[target].dtype == 'float64' or df[target].dtype == 'int64') and df[column].dtype == 'object'):    
            
            grouped_data = df.groupby(column)[target].apply(list)
            #print('ANOVA on ' + column)

            # Convert the grouped data into a list of arrays suitable for ANOVA
            data_arrays = [group for group in grouped_data]

            # Perform ANOVA
            if len(data_arrays) > 1:
                f, p = stats.f_oneway(*data_arrays)
                n = len(df.index)        # number of rows
                # Finding degrees of freedom for both numerator and demoninator, then using those to help find critical value
                dfn, dfd = (len(grouped_data) - 1), (n - 2)      # (2-1) because we are only comparing 2 groups, len(df.index) is number of rows
                critical_value = stats.f.ppf(1 - 0.05, dfn, dfd)
            else:
                correlation_categories["unable"].append(column)
                continue
                   
            
            if p < 0.05:
                if f > critical_value:
                    
                    print('Test: ANOVA and Eta-Squared (η²)')
                    
                    # Calculating the 3 values here that are needed to find the eta^2
                    # Calculate Total Sum of Squares (TSS)  
                    # all_data = np.concatenate(grouped_data)
                    all_data = []
                    for group in grouped_data:
                        all_data.extend(group)
                    all_data = np.array(all_data)
                    grand_mean = np.mean(all_data)
                    tss = np.sum((all_data - grand_mean) ** 2)

                    # Calculate Within-Group Sum of Squares (WSS)
                    wss = sum(sum((group - np.mean(group)) ** 2) for group in grouped_data)

                    # Calculate Between-Group Sum of Squares (BSS)
                    bss = tss - wss

                    # Calculate Eta-squared (η²)
                    eta_squared = bss / tss
                    
                    if eta_squared > 0.14:
                        doc = 'Large'
                        correlation_categories["large"].append(column)
                    elif eta_squared > 0.06:
                        doc = 'Medium'
                        correlation_categories["medium"].append(column)
                    elif eta_squared > 0.01:
                        doc = 'Small'
                        correlation_categories["small"].append(column)
                    else:
                        doc = 'None'
                        correlation_categories["none"].append(column)
                        
                    print(f'Target: {target} ({df[target].dtype})')
                    print(f'Column: {column} ({df[column].dtype})')
                    
                    if showGroupMeans == True:
                        print()
                        print('Table of Means by Each Group')
                        print()
                        group_means = df.groupby(column)[target].mean().reset_index()
                        print(round(group_means, 2))
                    
                    print()
                    print(f"f: {f:.3f}")
                    print(f"cv: {critical_value:.3f}", )
                    print(f"p: {p:.3f}")
                    print(f"Eta-Squared: {eta_squared:.3f}")     
                    print()
                    
                    print('Correlation:', doc)
                    
                    print()
                    #print('p value is below the significance threshold of 0.05')
                    #print('f value is greater than critical value')
                    #print('the difference between observed and expected frequencies is too large to be attributed to chance')

                    print('---------------------------------------------------------')
                    print()    
                    
                else:
                    correlation_categories["none"].append(column)
                    #print('No Correlation')
                    #print()
                    #print('p value is above the significance threshold of 0.05')
                    #print('f value is less than critical value')
                    #print('the difference between observed and expected frequencies is too small and could be attributed to chance')
                    
            else:
                correlation_categories["none"].append(column)
                #print('No Correlation')
                #print()
                #print('p value is above the significance threshold of 0.05')
                
                
        # numerical vs. object: Binned chi2 and Cramer's V (bias corrected)
        # using chi2 test, however I use bins that separate the numerical variable into a calculated number of bins
        # finding correlation between dependent categorical variable and independent continuous variable is quite
        # hard, and using chi2 with bins has been the most reasonable approach that I have been able to find
        elif (df[target].dtype == 'object' and (df[column].dtype == 'float64' or df[column].dtype == 'int64')):
            
            bin_edges = np.linspace(1, df[column].max(), num=bins+1)
            ct = pd.crosstab(df[target], pd.cut(df[column], bins=bin_edges, right=False, include_lowest=True), margins=False)
            #print(pd.cut(df[column], bin_edges).unique())
            
            chi2, p, dof, expected = stats.chi2_contingency(ct)
            critical_value = stats.chi2.ppf(1 - 0.05, dof)
            
            r, k = ct.shape                                      # Number of rows and columns
            n = np.sum(ct.values) 
            k_corrected_term = ((k - 1)**2) / (n - 1)            # Bias correction terms
            r_corrected_term = ((r - 1)**2) / (n - 1)
            k_corrected = k - k_corrected_term                   # Corrected number of rows and columns for bias correction
            r_corrected = r - r_corrected_term
                    
            if p < 0.05:
                if chi2 > critical_value:
                    
                    print('Test: Binned Chi-Square (Χ²) and Cramers V')
                    
                    print(f'Target: {target} ({df[target].dtype})')
                    print(f'Column: {column} ({df[column].dtype})')
                    print()
                    
                    cramers_v = np.sqrt((chi2 / n) / min(k_corrected - 1, r_corrected - 1))
    
                    if cramers_v > 0.5:
                        doc = 'Large'
                        correlation_categories["large"].append(column)
                    elif cramers_v > 0.3:
                        doc = 'Medium'
                        correlation_categories["medium"].append(column)
                    elif cramers_v > 0.1:
                        doc = 'Small'
                        correlation_categories["small"].append(column)
                    else:
                        doc = 'None'
                        correlation_categories["none"].append(column)

                    print(f'chi2: {chi2:.3f}')
                    print(f'cv: {critical_value:.3f}')          # this is the value that chi squared must be below or above to determine if it is high or low
                    print(f'p: {p:.3f}')
                    print(f'Cramers V: {cramers_v:.3f}')        # note that Cramers V works but is not suited towards 2x2 contingency tables
                    
                    print()    
                    print('Correlation:', doc)

                    print()
                    print('---------------------------------------------------------')
                    print()
                    
                    if showBinnedCrosstab == True:
                        print('Binned Crosstab:')
                        print()
                        print(ct)
                        print()
                    
                else: 
                    correlation_categories["none"].append(column)

            else:
                correlation_categories["none"].append(column)
            
    
    
# Printing out the final results, to see the strength of correlation of each variable
    
print('Correlation Between Target and Columns')
print()
print('Target:', target)
print()

    
# printing out the values in each category
for category, values in correlation_categories.items():
    if values:
        print(category.capitalize())
        for value in values:
            print(f" - {value}")
            
            
            

Test: Binned Chi-Square (Χ²) and Cramers V
Target: Quality (object)
Column: Sweetness (float64)

chi2: 78.444
cv: 68.669
p: 0.008
Cramers V: 0.302

Correlation: Medium

---------------------------------------------------------

Test: Binned Chi-Square (Χ²) and Cramers V
Target: Quality (object)
Column: Crunchiness (float64)

chi2: 102.028
cv: 65.171
p: 0.000
Cramers V: 0.226

Correlation: Small

---------------------------------------------------------

Test: Binned Chi-Square (Χ²) and Cramers V
Target: Quality (object)
Column: Ripeness (float64)

chi2: 112.303
cv: 68.669
p: 0.000
Cramers V: 0.267

Correlation: Small

---------------------------------------------------------

Correlation Between Target and Columns

Target: Quality

Medium
 - Sweetness
Small
 - Crunchiness
 - Ripeness
None
 - A_id
 - Size
 - Weight
 - Juiciness
 - Acidity


In [2]:
# This allows for the testing of the dataset to determine what datatypes have been automatically detected. 
# If a column appears as something it is not, please alter the data so that it can be properly processed 
# as the correct datatype. 

print()
for column in df.columns:
    print(f'{df[column].dtype} - {column}')


int64 - A_id
float64 - Size
float64 - Weight
float64 - Sweetness
float64 - Crunchiness
float64 - Juiciness
float64 - Ripeness
float64 - Acidity
object - Quality
